In [1]:
spark.sql('drop table tablex').collect()
spark.sql('create table tablex (a string,id string,part_id string,stime int) stored as parquet')



DataFrame[]

In [2]:
spark.sql('insert into tablex values("A","user1","part1",1),("B","user1","part1",1),("C","user1","part2",1),("A","user1","part2",1),("A","user1","part2",1),("A","user1","part26",1),("D","user1","part24",1),("Z","user1","part42",1),("A","user1","part22",1),("A","user12","part2",1)')


DataFrame[]

In [1]:
# df.show()
df=spark.table('tablex')
df.show()

+---+-----+-------+-----+
|  a|   id|part_id|stime|
+---+-----+-------+-----+
|  D|user1| part24|    1|
|  Z|user1| part42|    1|
|  B|user1|  part1|    1|
|  C|user1|  part2|    1|
|  D|user1| part24|    1|
|  Z|user1| part42|    1|
|  A|user1|  part1|    1|
|  D|user1| part24|    1|
|  Z|user1| part42|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
|  A|user1|  part2|    1|
+---+-----+-------+-----+
only showing top 20 rows



In [3]:
from pyspark.sql import functions as F
from pyspark.sql import types as T


In [7]:
df1=df.groupby('part_id','id').agg(F.collect_list("a").alias('ab') )
df1.show()

+-------+------+--------------------+
|part_id|    id|                  ab|
+-------+------+--------------------+
|  part2| user1|[C, A, A, A, A, A...|
| part26| user1|[A, A, A, A, A, A...|
|  part2|user12|[A, A, A, A, A, A...|
| part22| user1|[A, A, A, A, A, A...|
| part24| user1|[D, D, D, D, D, D...|
| part42| user1|[Z, Z, Z, Z, Z, Z...|
|  part1| user1|[A, B, A, B, A, A...|
+-------+------+--------------------+



# Output

In [8]:
# print(dir(F))
from pyspark.sql.functions import udf
from collections import Counter
import itertools

def prod(s):
    return list(map(lambda x:x[0]+x[1],list(itertools.product(s,s))))

prod1 = udf(prod,T.ArrayType(T.StringType()))


df2=df1.withColumn("ab", prod1("ab"))
df3=df2.withColumn('ab',F.explode('ab'))
df3.groupby('ab').count().show()


+---+-------+
| ab|  count|
+---+-------+
| AA|2097152|
| ZZ| 262144|
| DD| 262144|
| CA| 524288|
| CC| 262144|
| BA| 262144|
| AB| 262144|
| AC| 524288|
| BB| 262144|
+---+-------+



# Explain Plan

In [9]:
df3.withColumn("ab", prod1("ab")).explain()

== Physical Plan ==
*(3) Project [part_id#2, id#1, pythonUDF0#133 AS ab#128]
+- BatchEvalPython [prod(ab#106)], [part_id#2, id#1, ab#106, pythonUDF0#133]
   +- Generate explode(ab#101), [part_id#2, id#1], false, [ab#106]
      +- *(2) Project [part_id#2, id#1, pythonUDF0#132 AS ab#101]
         +- BatchEvalPython [prod(ab#84)], [part_id#2, id#1, ab#84, pythonUDF0#132]
            +- ObjectHashAggregate(keys=[part_id#2, id#1], functions=[collect_list(a#0, 0, 0)])
               +- Exchange hashpartitioning(part_id#2, id#1, 200)
                  +- ObjectHashAggregate(keys=[part_id#2, id#1], functions=[partial_collect_list(a#0, 0, 0)])
                     +- *(1) FileScan parquet default.tablex[a#0,id#1,part_id#2] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/home/arun/Spark/spark-warehouse/tablex], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<a:string,id:string,part_id:string>


In [12]:
spark.sql('insert into tablex select * from tablex')
spark.sql('select count(*) from tablex').collect()

[Row(count(1)=5120)]

In [21]:
print(dir(T))

['ArrayType', 'AtomicType', 'BinaryType', 'BooleanType', 'ByteType', 'CloudPickleSerializer', 'DataType', 'DataTypeSingleton', 'DateConverter', 'DateType', 'DatetimeConverter', 'DecimalType', 'DoubleType', 'FloatType', 'FractionalType', 'IntegerType', 'IntegralType', 'JavaClass', 'LongType', 'MapType', 'NullType', 'NumericType', 'Row', 'ShortType', 'SparkContext', 'StringType', 'StructField', 'StructType', 'TimestampType', 'UserDefinedType', '_FIXED_DECIMAL', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_acceptable_types', '_all_atomic_types', '_all_complex_types', '_array_signed_int_typecode_ctype_mappings', '_array_type_mappings', '_array_unsigned_int_typecode_ctype_mappings', '_atomic_types', '_check_dataframe_convert_date', '_check_dataframe_localize_timestamps', '_check_series_convert_date', '_check_series_convert_timestamps_internal', '_check_series_convert_timestamps_local_tz', '_check_series_convert_timest

# Using Join

In [4]:
df1=df.join(df.withColumnRenamed('a','b'),['id','part_id'],'left').withColumn('ab',F.concat(F.col('a'),F.col('b')))
df2=df1.select('ab').groupby('ab').agg(F.count('*').alias('Occurance'))

In [5]:
df2.explain()

== Physical Plan ==
*(6) HashAggregate(keys=[ab#55], functions=[count(1)])
+- Exchange hashpartitioning(ab#55, 200)
   +- *(5) HashAggregate(keys=[ab#55], functions=[partial_count(1)])
      +- *(5) Project [concat(a#0, b#40) AS ab#55]
         +- SortMergeJoin [id#1, part_id#2], [id#46, part_id#47], LeftOuter
            :- *(2) Sort [id#1 ASC NULLS FIRST, part_id#2 ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#1, part_id#2, 200)
            :     +- *(1) FileScan parquet default.tablex[a#0,id#1,part_id#2] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/home/arun/Spark/spark-warehouse/tablex], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<a:string,id:string,part_id:string>
            +- *(4) Sort [id#46 ASC NULLS FIRST, part_id#47 ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(id#46, part_id#47, 200)
                  +- *(3) Project [a#45 AS b#40, id#46, part_id#47]
                     +- *(3) Filte

In [6]:
df2.show()

+---+---------+
| ab|Occurance|
+---+---------+
| AA|  2097152|
| ZZ|   262144|
| DD|   262144|
| CA|   524288|
| CC|   262144|
| BA|   262144|
| AB|   262144|
| AC|   524288|
| BB|   262144|
+---+---------+

